In [1]:
import os, os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import re
import cv2

In [2]:
directory = 'MC2-Image-Data/'

In [50]:
file_list = []
caption_list = []
text_list = []
img_list = []
img_paths = []
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.csv'):
            file_list.append(os.path.abspath(root)+ '/' + file)
        if 'caption.txt' in file:
            caption_list.append(os.path.abspath(root)+ '/' + file)
        if 'text' in file:
            text_list.append(os.path.abspath(root) + '/' + file)
        if file.endswith('.jpg'):
            img_paths.append(os.path.abspath(root) + '/' + file)
            img_list.append(file)
# caption_list
img = pd.DataFrame(img_list)
img.columns =['Image']
img.head()

,Image
0,Person33_5.jpg
1,Person33_4.jpg
2,Person33_60.jpg
3,Person33_48.jpg
4,Person33_6.jpg


In [51]:
# img_paths

In [52]:
# img['PersonId'] = img['Image'].apply(lambda x: x.split('_')[0][6:])
img['ImageId'] = img['Image'].apply(lambda x: x.split('.')[0][6:])
img.head()

,Image,ImageId
0,Person33_5.jpg,33_5
1,Person33_4.jpg,33_4
2,Person33_60.jpg,33_60
3,Person33_48.jpg,33_48
4,Person33_6.jpg,33_6


In [53]:
# text_list

In [54]:
def read_csv(path):
    df = pd.read_csv(path, encoding= 'unicode_escape',dtype=str)
    df['PersonId'] = os.path.split(path)[1].split('_')[0][6:]
    df['ImageId'] = os.path.split(path)[1].split('.')[0][6:]
    return df

def read_txt(path, colName):
    df = pd.read_csv(path, sep='\t', encoding= 'unicode_escape', header=None, names=[colName])
    name = os.path.split(path)[1]
    if 'text' in name:
        df['TextId'] = name[6:-4]
    else:
#         df['PersonId'] = os.path.split(path)[1].split('_')[0][6:]
        df['ImageId'] = os.path.split(path)[1].split('.')[0][6:-7]
    return df

In [55]:
cap_list = []
txt_list = []
for file in caption_list:
    cap_df = read_txt(file, 'Caption')
    cap_list.append(cap_df)
for file in text_list:
    txt_list.append(read_txt(file, 'Text'))
cap = pd.concat(cap_list).reset_index(drop=True)
txt = pd.concat(txt_list).reset_index(drop=True)

In [56]:
cap.head()

,Caption,ImageId
0,Tortoise racing for life,33_42
1,Be the pop of color that can illuminate the mo...,33_17
2,Tortoise at Gym building Muscles,33_31
3,Rolling dice in a stable position,33_53
4,The things are not always where you think they...,33_1


In [57]:
txt.head()

,Text,TextId
0,You come home to find a single lit pink candle...,35_text3
1,Green turtles are a good investment Buy now!,35_text2
2,This trophy is a good prize for my win in HotS,35_text1
3,The benefit of the maple leaf pencil is that y...,35_text5
4,My cats enjoyed playing soccer with this eyeba...,35_text4


In [58]:
# im_cap = pd.merge(cap, img, how='outer', on='ImageId')
# im_cap

In [59]:
cap.to_json('caption.json', orient='records')
txt.to_json('text.json', orient='records')

In [60]:
df_list = []
for file in file_list:
    df = read_csv(file)
    df_list.append(df)
# df_list

In [61]:
concat_df = pd.concat(df_list).reset_index(drop=True)
concat_df.shape

(4491, 8)

In [62]:
concat_df.head(10)

,x,y,Width,Height,Score,Label,PersonId,ImageId
0,377,202,1237,1502,0.31465,cloudSign,33,33_40
1,2003,515,296,1361,0.48349,pinkCandle,33,33_40
2,229,273,2137,1825,0.34838,pumpkinNotes,33,33_40
3,1560,111,1210,2127,0.36762,sign,33,33_40
4,515,565,1341,2608,0.29639,birdCall,33,33_54
5,666,1143,1482,2514,0.29539,pumpkinNotes,33,33_54
6,797,1748,1200,1398,0.31314,redWhistle,33,33_54
7,11,269,1089,3750,0.47006,sign,33,33_54
8,1422,2366,1250,1680,0.3209,silverStraw,33,33_54
9,595,1143,1170,861,0.27061,blueSunglasses,33,33_54


In [64]:
columns = ['x','y','Width','Height','Score']
for col in columns:
    try:
        concat_df[col] = pd.to_numeric(concat_df[col], errors='coerce')
#         concat_df[col].astype('str').astype('int')

    except:
        pass

In [65]:
concat_df.dtypes

x           float64
y           float64
Width         int64
Height        int64
Score       float64
Label        object
PersonId     object
ImageId      object
dtype: object

In [126]:
# concat_df.to_json('data.json', orient='records')

In [ ]:
# df = pd.merge(concat_df, concat_txt, how='outer', on='PersonId')
# df[df['Label'].isnull()]

In [41]:
# df[df['x'].isnull()]

,x,y,Width,Height,Score,Label,PersonId


In [ ]:
# df

In [ ]:
# df.to_json('data_with_cap.json', orient='records')

In [50]:
d = concat_df.groupby(['PersonId'],as_index=False).agg({'Score': np.max })
d

,PersonId,Score
0,10_1,0.46997
1,10_10,0.40247
2,10_11,0.39960
3,10_12,0.51042
4,10_13,0.42838
...,...,...
889,9_5,0.65481
890,9_6,0.40167
891,9_7,0.59368
892,9_8,0.60256


In [129]:
labels = concat_df.groupby(['Label'],as_index=False).agg({'Score': np.max })['Label']
list(labels)

['birdCall',
 'blueSunglasses',
 'canadaPencil',
 'cloudSign',
 'cupcakePaper',
 'eyeball',
 'hairClip',
 'lavenderDie',
 'metalKey',
 'noisemaker',
 'paperPlate',
 'partyFavor',
 'pinkCandle',
 'pumpkinNotes',
 'redWhistle',
 'sign',
 'silverStraw',
 'stickerBox',
 'trophy',
 'vancouverCards',
 'yellowBag',
 'yellowBalloon']

In [17]:

def missing_ratio(df):
    '''
    INPUT: 
        - df : data frame to check
    OUTPUT: 
        - new_df : new dataframe with counts and ratio of missing values in each category
    '''              
    
    count = df.isnull().sum().sort_values(ascending = False)
    ratio = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    new_df = pd.concat([count, ratio], axis=1, keys=['Count', 'Ratio'])
    return new_df

In [19]:
missing_ratio(concat_df)

,Count,Ratio
ImageId,0,0.0
PersonId,0,0.0
Label,0,0.0
Score,0,0.0
Height,0,0.0
Width,0,0.0
y,0,0.0
x,0,0.0


In [52]:
concat_df['Label'].value_counts()

eyeball           501
cloudSign         482
sign              465
pinkCandle        415
blueSunglasses    352
birdCall          324
yellowBag         286
pumpkinNotes      271
hairClip          231
lavenderDie       200
redWhistle        181
cupcakePaper      135
metalKey          134
canadaPencil      116
stickerBox         92
partyFavor         91
trophy             68
silverStraw        58
yellowBalloon      30
noisemaker         26
paperPlate         22
vancouverCards     11
Name: Label, dtype: int64

In [20]:
person40 = concat_df[concat_df['PersonId'].str.startswith('40_')]
person40

,x,y,Width,Height,Score,Label,PersonId,ImageId


In [3]:
# im = cv2.imread('MC2-Image-Data/Person1/Person1_1.jpg')
# gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

In [6]:
# crop = gray[1310: 1586, 1502: 2258]
# cv2.imshow('test', crop)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [66]:
concat_df.head()

,x,y,Width,Height,Score,Label,PersonId,ImageId
0,377.0,202.0,1237,1502,0.31465,cloudSign,33,33_40
1,2003.0,515.0,296,1361,0.48349,pinkCandle,33,33_40
2,229.0,273.0,2137,1825,0.34838,pumpkinNotes,33,33_40
3,1560.0,111.0,1210,2127,0.36762,sign,33,33_40
4,515.0,565.0,1341,2608,0.29639,birdCall,33,33_54


In [89]:
concat_df[concat_df["x"].isnull()]

,x,y,Width,Height,Score,Label,PersonId,ImageId
237,NaN,NaN,2299,1360,0.85100,redWhistle,33,33_28
1245,NaN,NaN,1573,1544,0.86190,redWhistle,28,28_6
1246,NaN,NaN,1296,922,0.34825,hairClip,28,28_6
1336,NaN,NaN,2264,1671,0.33267,birdCall,26,26_6
1337,NaN,NaN,415,1141,0.26439,pinkCandle,26,26_6
...,...,...,...,...,...,...,...,...
4448,NaN,NaN,2160,1867,0.54900,pumpkinNotes,15,15_13
4449,NaN,NaN,1919,1521,0.32624,trophy,15,15_13
4450,NaN,NaN,2126,1152,0.46949,blueSunglasses,15,15_13
4451,NaN,NaN,1296,1279,0.46236,hairClip,15,15_13


In [90]:
concat_df[concat_df["y"].isnull()]

,x,y,Width,Height,Score,Label,PersonId,ImageId
237,NaN,NaN,2299,1360,0.85100,redWhistle,33,33_28
1245,NaN,NaN,1573,1544,0.86190,redWhistle,28,28_6
1246,NaN,NaN,1296,922,0.34825,hairClip,28,28_6
1336,NaN,NaN,2264,1671,0.33267,birdCall,26,26_6
1337,NaN,NaN,415,1141,0.26439,pinkCandle,26,26_6
...,...,...,...,...,...,...,...,...
4448,NaN,NaN,2160,1867,0.54900,pumpkinNotes,15,15_13
4449,NaN,NaN,1919,1521,0.32624,trophy,15,15_13
4450,NaN,NaN,2126,1152,0.46949,blueSunglasses,15,15_13
4451,NaN,NaN,1296,1279,0.46236,hairClip,15,15_13


In [91]:
concat_df[concat_df["Width"].isnull()]

,x,y,Width,Height,Score,Label,PersonId,ImageId


In [92]:
concat_df[concat_df["Height"].isnull()]

,x,y,Width,Height,Score,Label,PersonId,ImageId


In [67]:
concat_df[concat_df["ImageId"] == "33_1"].iloc[:, :4].values.tolist()

[[473.0, 197.0, 135.0, 406.0],
 [262.0, 5.0, 142.0, 278.0],
 [312.0, 871.0, 156.0, 376.0],
 [240.0, 13.0, 80.0, 244.0]]

In [88]:
concat_df[concat_df["ImageId"] == "33_1"][["x", "y", "Width", "Height", "Label"]]


,x,y,Width,Height,Label
313,473.0,197.0,135,406,cloudSign
314,262.0,5.0,142,278,eyeball
315,312.0,871.0,156,376,hairClip
316,240.0,13.0,80,244,lavenderDie


In [68]:
cordinates = concat_df[concat_df["ImageId"] == "33_1"][["x", "y", "Width", "Height", "Label"]].values.tolist()   
cordinates

[[473.0, 197.0, 135, 406, 'cloudSign'],
 [262.0, 5.0, 142, 278, 'eyeball'],
 [312.0, 871.0, 156, 376, 'hairClip'],
 [240.0, 13.0, 80, 244, 'lavenderDie']]

In [23]:
concat_df[concat_df["y"].isnull()]

,x,y,Width,Height,Score,Label,PersonId,ImageId


In [68]:
d = concat_df.set_index(['PersonId', 'Label'], drop=False)
d

x      y  Width  Height    Score         Label  \
PersonId Label                                                               
33_40    cloudSign      377.0  202.0   1237    1502  0.31465     cloudSign   
         pinkCandle    2003.0  515.0    296    1361  0.48349    pinkCandle   
         pumpkinNotes   229.0  273.0   2137    1825  0.34838  pumpkinNotes   
         sign          1560.0  111.0   1210    2127  0.36762          sign   
33_54    birdCall       515.0  565.0   1341    2608  0.29639      birdCall   
...                       ...    ...    ...     ...      ...           ...   
15_3     pumpkinNotes   416.0    6.0    464     662  0.25546  pumpkinNotes   
         sign           476.0  224.0    328    1382  0.61677          sign   
15_1     noisemaker       4.0    6.0    600     934  0.30253    noisemaker   
         pumpkinNotes   636.0  763.0    540     843  0.31760  pumpkinNotes   
         sign           900.0    6.0    304     955  0.34451          sign   

                      PersonId  
PersonId Label                  
33_40    cloudSign       33_40  
         pinkCandle      33_40  
         pumpkinNotes    33_40  
         sign            33_40  
33_54    birdCall        33_54  
...                        ...  
15_3     pumpkinNotes     15_3  
         sign             15_3  
15_1     noisemaker       15_1  
         pumpkinNotes     15_1  
         sign             15_1  

[4491 rows x 7 columns]

In [69]:
# d.to_json('h.json', orient='records')

In [69]:
def crop_image(path, x, y, w, h):
    """
    Function to crop an image based on cordinates of bounding box
    """
    img = cv2.imread(path)
    crop = img[y: y+h, x: x+w].copy()
    cv2.imwrite(save_location, crop)
    
    

In [64]:
crop_image('MC2-Image-Data/Person2/Person2_3.jpg', 1076, 474, 1035, 1049, 'MC2-Image-Data/test.jpg')
crop_image('MC2-Image-Data/Person1/Person1_2.jpg', 2984, 242,1062, 2712, 'MC2-Image-Data/test1.jpg')

In [70]:
img_paths

['/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_5.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_4.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_60.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_48.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_6.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_7.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_49.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_61.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_59.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_3.jpg',
 '/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/Person33/Person33_2.

In [93]:
def crop_image(path,df):
    """
    Function to crop an image based on cordinates of bounding box
    """
    img = cv2.imread(path)
    image_id = os.path.split(path)[1][6:-4]
    # Create a list of cordinate lists with [x, y, width, height]
    cordinates = concat_df[concat_df["ImageId"] == image_id][["x", "y", "Width", "Height", "Label"]].dropna().values.tolist()
    for cord in cordinates:
        x, y, w, h, label = int(cord[0]), int(cord[1]), int(cord[2]), int(cord[3]), cord[4]
        crop = img[y: y+h, x: x+w].copy()
        cv2.imwrite('/Users/jaywang/Documents/TTU_study/VAST2020_MC2/MC2-Image-Data/croped/{}-{}.jpg'.format(image_id, label ), crop)

In [94]:
for path in img_paths:
    crop_image(path, concat_df)